# Regression Part A


## 1) Explaination of the regression model

The goal of the regression is to predict the Symboling (risk factor) of a car based on its features, with the maximum accuracy possible.
In terms of feature transformation, we will use one-hot encoding for the categorical features (make, fuel-type, aspiration, num-of-doors, body-style, drive-wheels, engine-location, engine-type, num-of-cylinders, fuel-system) and standard scaling for the numerical features (normalized-losses, wheel-base, length, width, height, curb-weight, engine-size, bore, stroke, compression-ratio, horsepower, peak-rpm, city-mpg, highway-mpg, price).



In [9]:
import importlib_resources
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from scipy.io import loadmat
from sklearn import model_selection

from dtuimldmtools import rlr_validate


In [19]:
from data_load import getTargets
from data_standardization import X as X_num, missing_values as missing_values_num
from data_encoding import X as X_cat, missing_values as missing_values_cat
import pandas as pd
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from dtuimldmtools import rlr_validate
from data_load import categorical_features, numerical_features
import matplotlib.pyplot as plt



missing_values = list(set(missing_values_num).union(set(missing_values_cat)))

X = pd.concat([X_num, X_cat], axis=1)
y = getTargets()

X = X.drop(missing_values)
y = y.drop(missing_values)

X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

# Standardize X to make sure X_cat is also standardized, and to make sure that the mean and std of each column is 0 and 1 even without the rows with missing values
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Change X and y into a arrays
X = np.array(X)
y = np.array(y).squeeze()

N, M = X.shape






In [31]:
# Create a linear regression model
model = lm.LinearRegression(fit_intercept=True)

# Create a KFold object
K = 10
CV = model_selection.KFold(K, shuffle=True)

# Initialize variables
Error_train = np.empty((K, 1))
Error_test = np.empty((K, 1))
Error_train_rlr = np.empty((K, 1))
Error_test_rlr = np.empty((K, 1))

k = 0
for train_index, test_index in CV.split(X):
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]

    # Fit ordinary least squares regression model
    model = lm.LinearRegression(fit_intercept=True)
    model = model.fit(X_train, y_train)

    # Compute mean squared error without using the input data at all
    Error_train[k] = np.square(y_train - model.predict(X_train)).sum() / y_train.shape[0]
    Error_test[k] = np.square(y_test - model.predict(X_test)).sum() / y_test.shape[0]

    # Fit ridge regression model
    lambda_interval = np.logspace(-8, 2, 100)
    opt_val_err, opt_lambda, mean_w_vs_lambda, train_err_vs_lambda, test_err_vs_lambda = rlr_validate(
        X_train, y_train, lambda_interval, 10
    )

    # Compute mean squared error with ridge regression
    Error_train_rlr[k] = np.square(y_train - X_train @ mean_w_vs_lambda).sum() / y_train.shape[0]
    Error_test_rlr[k] = np.square(y_test - X_test @ mean_w_vs_lambda).sum() / y_test.shape[0]

    k = k + 1



ValueError: operands could not be broadcast together with shapes (180,) (180,100) 